In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz
!tar xf /content/spark-3.2.0-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.0-bin-hadoop3.2"


In [ ]:
import findspark
findspark.init()
from pyspark import SparkContext
from google.colab import files
files.upload()

Saving pagerank_bigdata.txt to pagerank_bigdata (5).txt


{'pagerank_bigdata (5).txt': b'n1 n2 n4\r\nn2 n3 n5 n6\r\nn3 n4 n7\r\nn4 n5\r\nn5 n1 n2 n3\r\nn6\r\nn7'}

In [ ]:
sc = SparkContext.getOrCreate()
rdd0 = sc.textFile("pagerank_bigdata.txt")
print(rdd0.collect())
def parseLinks(line):
  linelist = [x for x in line.split(" ")]
  return (linelist[0], linelist[1:])

['n1 n2 n4', 'n2 n3 n5 n6', 'n3 n4 n7', 'n4 n5', 'n5 n1 n2 n3', 'n6', 'n7']


In [ ]:
rdd1 = rdd0.map(lambda line : parseLinks(line))
partitionedrdd = rdd1.partitionBy(3, lambda key: hash(key))
print(rdd1.collect())

[('n1', ['n2', 'n4']), ('n2', ['n3', 'n5', 'n6']), ('n3', ['n4', 'n7']), ('n4', ['n5']), ('n5', ['n1', 'n2', 'n3']), ('n6', []), ('n7', [])]


In [ ]:
alpha=0.75
N=rdd1.count()
initialpagerank=1.0/N
delta=0
threshold = 0.0001
pagerankrdd = rdd1.map(lambda line : (line[0], initialpagerank))
print(pagerankrdd.collect())

[('n1', 0.14285714285714285), ('n2', 0.14285714285714285), ('n3', 0.14285714285714285), ('n4', 0.14285714285714285), ('n5', 0.14285714285714285), ('n6', 0.14285714285714285), ('n7', 0.14285714285714285)]


In [ ]:
def evalpagerank(line,deltaval):
  initialpagerank= (alpha/N) + (deltaval/N + line[1])*(1 - alpha)
  return ((line[0], initialpagerank))


def findpagerank(node):
    returnlist = []
    parent_node, parent_node_rank_value, child_node_list = node[0], node[1][0], node[1][1]
    no_of_child_nodes = len(child_node_list)
    for child_node in child_node_list:
        child_rank_value = parent_node_rank_value / no_of_child_nodes
        returnlist.append((child_node, child_rank_value))
    return returnlist

In [ ]:
from operator import add, sub
def calculate_abs_diff(line):
    return line

def filter_above_threshold(line):
    return abs(line[1]) > threshold

def isconvergent(prevrdd, newrdd):
    global threshold
    abs_diff = (prevrdd.union(newrdd)).map(calculate_abs_diff).reduceByKey(sub)
    abovethreshold = abs_diff.filter(filter_above_threshold)
    if abovethreshold.count() < 1:
        return abs_diff, 1
    else:
        return abs_diff, 0

In [ ]:
'''
Dangle node section
'''
def checkdelta(line):
  if (len(line[1][1])==0):
    return line[1][0]
  else:
    return 0

In [ ]:
i=1
while (i<25):
  print("\nPage Ranks in",i,"th iteration: ",pagerankrdd.collect())
  mergedrdd = pagerankrdd.join(partitionedrdd)
  newdelta = mergedrdd.map(lambda line: checkdelta(line)).reduce(add)
  print("New Delta:",newdelta)
  rankvaluesofrdd = mergedrdd.flatMap(lambda line: findpagerank(line)).reduceByKey(add)
  oldranks = pagerankrdd
  pagerankrdd = rankvaluesofrdd.map(lambda line: evalpagerank(line,delta))
  abslist, convergence = isconvergent(oldranks, pagerankrdd)
  print("Absolute difference:",abslist.collect())
  if convergence:
    break
  i=i+1


Page Ranks in 1 th iteration:  [('n1', 0.14285714285714285), ('n2', 0.14285714285714285), ('n3', 0.14285714285714285), ('n4', 0.14285714285714285), ('n5', 0.14285714285714285), ('n6', 0.14285714285714285), ('n7', 0.14285714285714285)]
New Delta: 0.2857142857142857
Absolute difference: [('n1', 0.023809523809523808), ('n3', 0.011904761904761918), ('n4', 0.0), ('n5', -0.011904761904761918), ('n7', 0.01785714285714285), ('n6', 0.023809523809523808), ('n2', 0.005952380952380959)]

Page Ranks in 2 th iteration:  [('n6', 0.11904761904761904), ('n1', 0.11904761904761904), ('n3', 0.13095238095238093), ('n4', 0.14285714285714285), ('n7', 0.125), ('n5', 0.15476190476190477), ('n2', 0.1369047619047619)]
New Delta: 0.24404761904761904
Absolute difference: [('n5', 0.0004960317460317443), ('n7', 0.0014880952380952467), ('n2', 0.001984126984126977), ('n6', 0.0004960317460317443), ('n1', -0.0009920634920634885), ('n3', -0.000496031746031772), ('n4', 0.004464285714285726)]

Page Ranks in 3 th iteration

In [ ]:
sortedrdd = pagerankrdd.sortBy(lambda x:x[1])
print("Final page ranks:",sortedrdd.collect())
print("Is Convergence found?",convergence)

Final page ranks: [('n6', 0.11838380341527042), ('n1', 0.11989601130768665), ('n7', 0.12353558683311286), ('n3', 0.13113695758009994), ('n2', 0.13488245655129336), ('n4', 0.13852203207671956), ('n5', 0.15302077018114343)]
Is Convergence found? 1
